<a href="https://colab.research.google.com/github/moshms313-a11y/Biki/blob/main/Foodics_Backend_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


GridBox(children=(Dropdown(layout=Layout(width='auto'), options=('google/gemini-2.5-flash', 'google/gemini-2.5…

In [ ]:
import csv
import re
import json
import random

# --- AI & Intelligence Layer ---

class GeminiIntelligence:
    """
    AI Logic Module.
    Designed to connect with Google Gemini API for semantic auditing and content generation.
    """
    def __init__(self, api_key=None):
        self.api_key = api_key
        # Simulation mode is active if no API key is provided
        self.simulation_mode = api_key is None

    def semantic_audit(self, product_name, category, price):
        """
        AI Analysis: Checks if the product makes sense in its category and price range.
        Example: Detecting 'Pepsi' in 'Burgers' category.
        """
        if self.simulation_mode:
            # Simple heuristic simulation for immediate feedback without API key
            name_lower = str(product_name).lower()
            cat_lower = str(category).lower()

            # Logic: Detect mismatched categories (e.g., Drink in Burger category)
            drinks_keywords = ['pepsi', 'cola', 'water', 'juice', 'coffee']
            if any(k in name_lower for k in drinks_keywords) and 'burger' in cat_lower:
                return {"flag": True, "reason": "AI Alert: Beverage detected in 'Burger' category."}

            # Logic: Price anomaly detection
            try:
                p = float(price)
                if p > 500 and 'sandwich' in name_lower:
                    return {"flag": True, "reason": "AI Alert: Price seems unusually high for a sandwich."}
            except:
                pass

            return {"flag": False, "reason": "OK"}
        else:
            # TODO: Integration with real Gemini API
            # payload = f"Analyze if '{product_name}' fits in '{category}' with price {price}."
            # return call_gemini_api(payload)
            pass

    def generate_content(self, product_name, field_type):
        """
        Generates missing content (Descriptions, Arabic translations).
        """
        if self.simulation_mode:
            if field_type == "Description":
                return f"Delicious {product_name} prepared with fresh ingredients."
            elif field_type == "Name (Ar)":
                return f"[AI Translated] {product_name}"
        return ""

class MenuEngineeringAI:
    """
    Analyzes profitability and menu structure based on Foodics PDF best practices.
    """
    def analyze_profitability(self, cost, price):
        try:
            c = float(cost)
            p = float(price)
            if p == 0: return "Warning: Selling Price is 0"
            margin = ((p - c) / p) * 100

            if margin < 20:
                return f"Low Margin ({margin:.1f}%). Consider re-engineering recipe or raising price."
            elif margin > 85:
                return f"High Margin ({margin:.1f}%). Good 'Cash Cow' candidate."
            return "Healthy Margin"
        except:
            return "Invalid Data"

# --- Google Sheets Bridge ---

class GoogleSheetsAdapter:
    """
    Bridge to format data for Google Sheets input/output.
    """
    @staticmethod
    def to_sheet_row(headers, row_dict):
        """Converts a dict to a list of values based on headers order."""
        return [row_dict.get(h, "") for h in headers]

    @staticmethod
    def from_sheet_row(headers, row_values):
        """Converts a list of values to a dict based on headers."""
        return dict(zip(headers, row_values))

    @staticmethod
    def generate_batch_update(data_report):
        """Creates a payload to color-code invalid cells in Sheets."""
        requests = []
        for issue in data_report:
            row_idx = issue['row'] - 1 # 0-indexed
            requests.append({
                "repeatCell": {
                    "range": {
                        "startRowIndex": row_idx,
                        "endRowIndex": row_idx + 1,
                        "startColumnIndex": 0,
                        "endColumnIndex": 1
                    },
                    "cell": {"userEnteredFormat": {"backgroundColor": {"red": 1, "green": 0.8, "blue": 0.8}}},
                    "fields": "userEnteredFormat.backgroundColor"
                }
            })
        return requests

# --- Core Logic Classes (Existing + Enhanced) ---

class BaseTemplateLogic:
    """Base logic for all Foodics templates."""
    def __init__(self):
        self.headers = []
        self.required_fields = []

    def validate_headers(self, file_headers):
        missing = [h for h in self.headers if h not in file_headers]
        if missing:
            return False, f"Missing headers: {', '.join(missing)}"
        return True, "Headers valid"

    def generate_template(self):
        return ",".join(self.headers)

class ProductLogic(BaseTemplateLogic):
    def __init__(self):
        super().__init__()
        self.headers = [
            "Name", "Name (Ar)", "Description", "Description (Ar)", "SKU", "Barcode",
            "Category Reference", "Is Selling Item", "Is Stock Item", "Is Modifier",
            "Pricing Method", "Price", "Cost Method", "Cost", "Tax Group Reference",
            "Calories", "Image URL"
        ]
        self.required_fields = ["Name", "SKU", "Category Reference", "Price"]

    def validate_row(self, row, ai_engine=None):
        errors = []
        warnings = []

        # 1. Hard Logic Validation
        if not row.get("SKU"): errors.append("SKU is mandatory.")
        if row.get("Pricing Method") not in ["Fixed", "Open"]: errors.append("Pricing Method must be 'Fixed' or 'Open'.")

        if row.get("Is Stock Item") == "0" and row.get("Cost Method") == "Fixed" and float(row.get("Cost", 0)) == 0:
            warnings.append("Warning: Cost is Fixed at 0 for a recipe item.")

        # 2. AI & Menu Engineering Validation
        if ai_engine:
            # Profitability Check
            profit_msg = ai_engine['menu'].analyze_profitability(row.get("Cost", 0), row.get("Price", 0))
            if "Low Margin" in profit_msg: warnings.append(profit_msg)

            # Semantic Check
            semantic = ai_engine['gemini'].semantic_audit(row.get("Name"), row.get("Category Reference"), row.get("Price"))
            if semantic['flag']:
                warnings.append(semantic['reason'])

        return len(errors) == 0, errors, warnings

class InventoryLogic(BaseTemplateLogic):
    def __init__(self):
        super().__init__()
        self.headers = [
            "Name", "Name (Ar)", "SKU", "Storage Unit", "Ingredient Unit",
            "Conversion Factor", "Cost", "Cost Method", "Reorder Level", "Barcode"
        ]

    def validate_row(self, row, ai_engine=None):
        errors = []
        warnings = []

        try:
            factor = float(row.get("Conversion Factor", 0))
            if factor <= 0: errors.append("Conversion Factor must be > 0.")
        except: errors.append("Conversion Factor must be a number.")

        if row.get("Storage Unit") == row.get("Ingredient Unit") and float(row.get("Conversion Factor", 1)) != 1:
             errors.append("If units match, Conversion Factor must be 1.")

        return len(errors) == 0, errors, warnings

class RecipeLogic(BaseTemplateLogic):
    def __init__(self):
        super().__init__()
        self.headers = ["Product SKU", "Ingredient SKU", "Quantity"]

    def validate_row(self, row, ai_engine=None):
        errors = []
        try:
            if float(row.get("Quantity", 0)) <= 0: errors.append("Quantity must be > 0.")
        except: errors.append("Quantity must be a number.")
        return len(errors) == 0, errors, []

class ModifierLogic(BaseTemplateLogic):
    def __init__(self):
        super().__init__()
        self.headers = [
            "Modifier Set Name", "Modifier Set Reference", "Option Name",
            "Option Name (Ar)", "Price", "Tax Group Reference", "Is Default"
        ]

    def validate_row(self, row, ai_engine=None):
        return True, [], []

class ComboLogic(BaseTemplateLogic):
    def __init__(self):
        super().__init__()
        self.headers = [
            "Combo Name", "Combo SKU", "Group Name", "Option Product SKU", "Price Adjustment"
        ]

    def validate_row(self, row, ai_engine=None):
        return True, [], []

# --- Master Engine ---

class FoodicsMasterEngine:
    """
    Master Controller.
    Integrates Validation Logic, AI Analysis, and Sheets Adapters.
    """
    def __init__(self, gemini_api_key=None):
        self.logics = {
            "product": ProductLogic(),
            "inventory": InventoryLogic(),
            "recipe": RecipeLogic(),
            "modifier": ModifierLogic(),
            "combo": ComboLogic()
        }
        # Initialize AI Engines
        self.gemini = GeminiIntelligence(api_key=gemini_api_key)
        self.menu_engineer = MenuEngineeringAI()
        self.sheets_adapter = GoogleSheetsAdapter()

    def analyze_file(self, file_type, file_content_str):
        """
        Analyzes a raw CSV string with enabled AI features.
        """
        if file_type not in self.logics:
            return {"status": "error", "message": "Unknown template type"}

        logic = self.logics[file_type]
        rows = []
        try:
            reader = csv.DictReader(file_content_str.splitlines())
            rows = list(reader)
        except Exception as e:
            return {"status": "error", "message": f"CSV Parsing Failed: {str(e)}"}

        header_valid, header_msg = logic.validate_headers(reader.fieldnames)
        if not header_valid:
            return {"status": "error", "message": header_msg}

        report = []
        ai_engines = {'gemini': self.gemini, 'menu': self.menu_engineer}

        for index, row in enumerate(rows):
            # Pass AI engines to validate_row
            is_valid, errors, warnings = logic.validate_row(row, ai_engine=ai_engines)

            if not is_valid or warnings:
                report.append({
                    "row": index + 2,
                    "status": "Invalid" if errors else "Warning",
                    "errors": errors,
                    "warnings": warnings,
                    "ai_suggestions": self._get_ai_suggestions(row, file_type) if not is_valid or warnings else []
                })

        return {
            "status": "success",
            "total_rows": len(rows),
            "issues_found": len(report),
            "report": report
        }

    def _get_ai_suggestions(self, row, file_type):
        """Internal helper to ask Gemini for specific fixes."""
        suggestions = []
        if file_type == "product":
            if not row.get("Name (Ar)"):
                suggestions.append(f"Auto-translate name: {self.gemini.generate_content(row.get('Name'), 'Name (Ar)')}")
            if not row.get("Description"):
                suggestions.append(f"Generate description: {self.gemini.generate_content(row.get('Name'), 'Description')}")
        return suggestions

    def get_template_headers(self, file_type):
        if file_type in self.logics:
            return self.logics[file_type].generate_template()
        return ""

    def process_from_google_sheets(self, file_type, raw_values):
        """
        Directly process 2D array data from Google Sheets API.
        """
        if not raw_values: return {"status": "empty"}
        headers = raw_values[0]
        data_rows = raw_values[1:]

        # Convert to Dict for internal logic
        csv_like_rows = [dict(zip(headers, r)) for r in data_rows]

        # Re-use analysis logic (simulated by converting back to string or refactoring)
        # For efficiency, we would refactor analyze_file to accept list of dicts.
        # Here we simulated the output structure for the user.
        return self.analyze_file(file_type, self._dict_list_to_csv_str(headers, csv_like_rows))

    def _dict_list_to_csv_str(self, headers, rows):
        import io
        output = io.StringIO()
        writer = csv.DictWriter(output, fieldnames=headers)
        writer.writeheader()
        writer.writerows(rows)
        return output.getvalue()

# --- Usage Example ---
if __name__ == "__main__":
    # Initialize with simulated AI
    engine = FoodicsMasterEngine()

    print("--- 1. Generating Smart Template ---")
    print(engine.get_template_headers("product"))

    # Simulate a user uploading a CSV with logical errors
    dummy_csv_with_errors = """Name,SKU,Pricing Method,Is Stock Item,Cost Method,Cost,Price,Category Reference,Name (Ar),Description
    Pepsi,DRK-001,Fixed,1,Fixed,2,5,Burgers,,
    Super Burger,BRG-999,Fixed,0,From Ingredients,15,16,Burgers,,"""

    print("\n--- 2. Running AI Audit ---")
    result = engine.analyze_file("product", dummy_csv_with_errors)
    print(json.dumps(result, indent=2))

--- 1. Generating Smart Template ---
Name,Name (Ar),Description,Description (Ar),SKU,Barcode,Category Reference,Is Selling Item,Is Stock Item,Is Modifier,Pricing Method,Price,Cost Method,Cost,Tax Group Reference,Calories,Image URL

--- 2. Running AI Audit ---
{
  "status": "error",
  "message": "Missing headers: Description (Ar), Barcode, Is Selling Item, Is Modifier, Tax Group Reference, Calories, Image URL"
}
